In [1]:
import tensorflow as tf
import numpy as np
import pickle as pkl
from sklearn import preprocessing as pre
#from pathos.pools import ProcessPool as Pool
tf.set_random_seed(1234)
np.random.seed(1234)
import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [3]:
#some hypers
GO='<START>'
STOP='<END>'
pad='<PAD>'
unknown='<UNKNOWN>'
BATCH=64
EPOCHS=50
embedding_size=512
lstm_units=1024
dropout_keep_prob=0.5


In [4]:
#load data
with open('./lingua.pkl') as f:
    trainCaptions=pkl.load(f)
trainCaptions=trainCaptions[:100000]

In [5]:
def processToken(caps):
    nk=[]
    
    for i in caps:
        t=[GO]+list(i)+[STOP]
        nk.append(t)
    return nk

In [6]:
trainCaptions=processToken(trainCaptions)


In [7]:
def listofwords(data):
    '''takes a list of sentences nd returns vocab'''
    a=[]
    for i in data:
        for j in i:
            if j not in a:
                a.append(j)
    return a

In [8]:
#load data
#with open('./vocab.pkl') as f:
#    trainVocab=pkl.load(f)

trainVocab=listofwords(trainCaptions)

In [9]:
#Label Encoder for output transform
pre_op=pre.LabelEncoder()
pre_op.fit(trainVocab)
onehoter=np.identity(len(pre_op.classes_),dtype=np.bool)


In [10]:
#word to int
from tqdm import tqdm_notebook
trainSeq=[]
for i in tqdm_notebook(range(len(trainCaptions))):
    trainSeq.append(pre_op.transform(trainCaptions[i]))

In [11]:
trainSeqLen=[len(i) for i in tqdm_notebook(trainSeq)]

with open('./vocab.pkl','w+') as f:
    pkl.dump(obj=trainVocab,file=f)

In [38]:
#aeppending stops

MAX_LEN=max(trainSeqLen)
trainSeqReg=[np.pad(i,(0,MAX_LEN-len(i)),'constant',constant_values=pre_op.transform([STOP])) for i in tqdm_notebook(trainSeq)]
trainSeqReg=np.array(trainSeqReg)


In [39]:
#Lets build the graph
tf.reset_default_graph()

In [40]:
source_seq = tf.placeholder(shape=(None,None),dtype=tf.int32)
source_seq_len = tf.placeholder(shape=(None,), dtype=tf.int32)
no_start_target_seq = tf.placeholder(shape=(None,None),dtype=tf.int32)
real_target_seq_len= tf.placeholder(shape=(None,), dtype=tf.int32)
batch_size = tf.placeholder(shape=(None),dtype=tf.int32)
keep_prob= tf.placeholder(dtype=tf.float32)

In [41]:
#output embeddings
embedding_matrix_encode = tf.get_variable(
    name="embedding_matrix_en",
    shape=[len(trainVocab), embedding_size],
    dtype=tf.float32)
decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_encode, source_seq) 


#output embeddings
embedding_matrix_decode = tf.Variable(initial_value=tf.random_normal(shape=[trainVocabSize, embedding_size],dtype=tf.float32))
decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_decode, target_seq) 


In [42]:
#encoder
encoderCell=tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_units),input_keep_prob=keep_prob,
                                          output_keep_prob=keep_prob)
#encoder_outputs,encoder_final_state=tf.nn.dynamic_rnn(cell=encoderCell,inputs=source_seq,sequence_length=source_seq_len,
#                dtype=tf.float32)



In [43]:
#Training helper and decoder
output_layer = tf.layers.Dense(len(trainVocab))
helper = tf.contrib.seq2seq.TrainingHelper(decoder_input_embedded,source_seq_len)
decoder = tf.contrib.seq2seq.BasicDecoder(encoderCell, helper,encoderCell.zero_state(BATCH,dtype=tf.float32),output_layer=output_layer)#,output_layer=projection_layer)
outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder)
logits = outputs.rnn_output


In [44]:
#loss and optimizer

#loss1
cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=no_start_target_seq,logits=logits)

target_weights = tf.sequence_mask(real_target_seq_len, source_seq_len[0], dtype=logits.dtype)

loss=tf.reduce_sum(cross_entropy*target_weights)

#train = tf.train.AdamOptimizer().minimize(total_loss)

#gradient clipping stackoverflow
optimizer = tf.train.AdamOptimizer()
gradients, variables = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(gradients, 10.0)
train = optimizer.apply_gradients(zip(gradients, variables))


In [45]:
#dont touch
maxtlen=max(trainSeqLen)
t_newlen=[maxtlen-1 for i in range(len(trainSeqLen))]

In [46]:
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

In [47]:
saver=tf.train.Saver()

In [48]:
#makes training batch
def getTrainBatch(indexs):
    targetBatch=np.array([trainSeqReg[i] for i in indexs])
    targetBatchLen=np.array([trainSeqLen[i] for i in indexs])
    return targetBatch,targetBatchLen



#restore model
saver.restore(sess, "BestModel/model.ckpt")

In [ ]:
#training starts here

print 'starting training'
training_losses=[]
valid_losses=[]
tData=np.arange(len(trainSeqReg))
for j in range(EPOCHS):
    np.random.shuffle(tData) #makes them iid
    training_loss=0
    for i in range(len(trainSeqReg)/BATCH):
        start=i*BATCH
        stop=(i+1)*BATCH
        targetBatch,targetBatchLen=getTrainBatch(tData[start:stop])
        _,lost=sess.run([train,loss],feed_dict={source_seq:targetBatch,
                                                
                                              source_seq_len:t_newlen[start:stop],
                                                real_target_seq_len:targetBatchLen,
                                                no_start_target_seq:np.array(targetBatch)[:,1:],
                                                batch_size:BATCH,keep_prob:dropout_keep_prob
                                                })
        
        training_loss+=lost
        #print lost,
    #calculate t_loss
    training_losses.append(training_loss/len(trainSeqReg))
    
    print "Epoch:%d training loss:%.4f"% (j,training_losses[-1])
    

starting training


In [ ]:
if PATIENCE_MONITOR:
    saver.restore(sess, "model/bestModel.ckpt")

In [ ]:
#save model
saver.save(sess, "model/continue.ckpt")

In [ ]:
data=videoFeats[1300:]
data_len=videoFeatSize[1300:]
if BATCH==1024:
    data=np.concatenate([data,data[:354]])
    data_len=np.concatenate([data_len,data_len[:354]])
gen_sum=[]
for i in range(len(data)/BATCH):
    start=i*BATCH
    stop=(i+1)*BATCH
    
    load_trs=trs_beam
    y=sess.run(load_trs,feed_dict={source_seq:data[start:stop],
                                               source_seq_len:data_len[start:stop],
                                              batch_size:BATCH,keep_prob:1.0
                                                })
    y=y[:,:,0]
    
    for t in y:
        gen_sum.append(t)

'''        
start=len(data)-BATCH
stop=len(data)
y=sess.run(trs_beam,feed_dict={source_seq:data[start:stop],
                                               source_seq_len:data_len[start:stop],
                                              batch_size:BATCH,keep_prob:1.0
                                                })
y=y[:,:,0]

y=y[-(len(data)-len(gen_sum)):]
for t in y:
    gen_sum.append(t)
'''

In [ ]:
gen_sum=gen_sum[:670]

data=videoFeats[1300:1364]
data_len=videoFeatSize[1300:1364]
gen_sum=[]
for i in range(len(data)/BATCH):
    start=i*BATCH
    stop=(i+1)*BATCH
    load_trs=trs
    y=sess.run(load_trs,feed_dict={source_seq:data[start:stop],
                                               source_seq_len:data_len[start:stop],
                                              batch_size:BATCH,
                                                keep_prob:1.0
                                                })
    for t in y:
        gen_sum.append(t)


In [ ]:
#processing summaries
summs=[]
for i in gen_sum:
    summ=''
    for j in i:
        
        if j!=58:
            summ = summ+' '+pre_op.inverse_transform(j)
    summs.append(summ[1:])

for i in range(len(summs)):
    print i,summs[i],vops[i]

vops=np.load('../MSVD/clip_index/testIndex.npy')[:64]
ref_test

#real shit
with open('ref_dev.txt','w+') as fle:
    for i in devCaptions:
        fle.write('vid'+str(i[0])+'\t'+' '.join(i[1][1:-1]))
        fle.write('\n')

In [ ]:
vdo=1300
with open('gen.txt','w+') as fle:
    for i in summs:
        fle.write('beam_size_5'+'\tvid'+str(vdo)+'\t'+i)
        fle.write('\n')
        vdo+=1